In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pickle
import numpy as np
import time
import math
import string
%matplotlib inline

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

use_cuda = torch.cuda.is_available()
print(" I have a GPU? ", use_cuda)

 I have a GPU?  False


## 1. smaller architecture

## 2. validation set

## 3. validation evaluator 

In [2]:
# This is a Character Level RNN, it used the most common symbols and the upper and lower case Letters

char2idx, idx2char, character_list = pickle.load( open( "data/characters.p", "rb" ) )

pairs = pickle.load( open( "data/8pairs_50_1000.p", "rb" ) ) 

print(len(pairs))

training_pairs = pairs[:25000]
validation_pairs = pairs[25000:]

class2index = {
 'addiction': 2,
 'anxiety': 4,
 'autism': 0,
 'bipolar': 7,
 'conversation': 3,
 'depression': 6,
 'happy': 5,
 'schizophrenia': 1,
}

index2class =  {v: k for k, v in class2index.items()}

''' 
pairs = [
['sad','depression'],    
['lonely','depression'],
['stress','anxiety'],
['happy','happy'],
['love','happy'],    
]



new_pairs = []
for pair in pairs:
    if len(pair[0]) > 300 and len(pair[0]) < 310:
        new_pairs.append(pair)
pairs = new_pairs
len(pairs)
'''

25604


" \npairs = [\n['sad','depression'],    \n['lonely','depression'],\n['stress','anxiety'],\n['happy','happy'],\n['love','happy'],    \n]\n\n\n\nnew_pairs = []\nfor pair in pairs:\n    if len(pair[0]) > 300 and len(pair[0]) < 310:\n        new_pairs.append(pair)\npairs = new_pairs\nlen(pairs)\n"

In [3]:
def indexesFromSentence(sentence):
    '''
     | is our end of sentence character 
     ~ is our start of sentence character
     if these occur naturally in our string, replace it with our unknown character
     _ is our unknown character
    '''
    sentence_as_indices = []
    sentence = sentence.replace("|","_")
    sentence = sentence.replace("~","_")
    for char in sentence:
        if char in char2idx:
            sentence_as_indices.append(char2idx[char])
        else:
            sentence_as_indices.append(char2idx["_"])
            
    sentence_as_indices.append(char2idx["|"])
    
    return sentence_as_indices

def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))


In [4]:
# Pad a with the PAD symbol
def pad_seq(seq, max_length):
    seq += [char2idx[" "] for i in range(max_length - len(seq))]
    return seq

def random_batch(batch_size):
    input_seqs = []
    targets = []

    # Choose random pairs
    for i in range(batch_size):
        #pair = random.choice(pairs)
        pair = random.choice(training_pairs)
        input_seqs.append(indexesFromSentence(pair[0]))
        targets.append(class2index[pair[1]])

    # Zip into pairs, sort by length (descending), unzip
    #seq_pairs = sorted(zip(input_seqs, target_seqs), key=lambda p: len(p[0]), reverse=True)
    #input_seqs, target_seqs = zip(*seq_pairs)
    
    # For input and target sequences, get array of lengths and pad with 0s to max length
    input_lengths = [len(s) for s in input_seqs]
    input_padded = [pad_seq(s, max(input_lengths)) for s in input_seqs]
    #target_lengths = [len(s) for s in target_seqs]
    #target_padded = [pad_seq(s, max(target_lengths)) for s in target_seqs]

    # Turn padded arrays into (batch_size x max_len) tensors, transpose into (max_len x batch_size)
    input_var = Variable(torch.LongTensor(input_padded)).transpose(0, 1)
    #target_var = Variable(torch.LongTensor(target_padded)).transpose(0, 1)
    targets = Variable(torch.LongTensor(targets))
    
    if use_cuda:
        input_var = input_var.cuda()
        targets = targets.cuda()
        
    return input_var, input_lengths, targets

In [5]:
input_var, input_lengths, targets = random_batch(2)
input_var

Variable containing:
   14     0
   81    49
   41    50
   48    38
   44    50
   81    81
   42    28
   49    30
   51    81
   39    45
   27    30
   51    43
   39    81
   46    42
   81    49
   48    51
   33    39
   30    27
   43    81
   42    36
   81    30
   32    44
   43    42
   36    81
   26    30
   51    35
   39    81
   46    45
   81    30
   30    43
   35    38
   35    81
   81    48
   42    39
   49    29
   51    51
   44    50
   81    42
   33    45
   38    81
   51    47
   28    30
   46    36
   50    50
   55    44
   81    81
   14    35
   42    38
   81    30
   47    36
   48    56
   36    81
   50     2
   81    39
   43    28
   26    81
   81    51
   50    35
   48    81
   38    45
   40    30
   45    43
   81    81
   30    41
   39    48
   81    39
   36    42
   50    81
   81    42
   48    30
   39    81
   28    48
   81    39
   14    44
   81    41
   32    50
   43    38
   44    81
   42    42
   81    49
   27    51
   51  

In [6]:
indexesFromSentence("hi ` you")

[49, 51, 81, 104, 81, 45, 30, 43, 60]

In [7]:
class EncoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers = 3, bidirectional = False, dropout=0.1):
        
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, 
                          dropout=self.dropout, bidirectional=bidirectional)
        
        if bidirectional:
            num_directions = 2
        else:
            num_directions = 1
        
        hidden0 = torch.zeros(self.num_layers*num_directions, 1, self.hidden_size)
        self.hidden0 = nn.Parameter(hidden0, requires_grad=True)

    def forward(self, input_seqs):
        
        # expects inputs of shape [torch.cuda.LongTensor of size seq_length x batch_size ]
        
        if use_cuda:
            input_seqs.cuda()
            
        batch_size = input_seqs.size(1)
        hidden = self.hidden0.repeat(1, batch_size, 1)
        self.embedded = self.embedding(input_seqs)
        output, hidden = self.gru(self.embedded, hidden)

        if self.bidirectional:
            # Sum bidirectional outputs
            output = output[:, :, :self.hidden_size] + output[:, : ,self.hidden_size:] 
          
        return output, hidden  # hidden.shape = [layers x directions, batch_size, hidden_size]
    
    def initHidden(self):
        
        if use_cuda:
            return self.hidden0.cuda()
        else:
            return self.hidden0

In [8]:
class DecoderRNN(nn.Module):
    
    def __init__(self, example_encoder_hidden_state, num_classes,vector_size):
        
        super(DecoderRNN, self).__init__()
        
        # hidden.shape = [layers x directions, batch_size, hidden_size]
        # num_cells = layers x directions
        num_cells, batch_size, hidden_size = encoder_hidden.shape
        self.num_cells = num_cells
        self.batch_size = batch_size 
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.vector_size = vector_size
        
        self.fc1 = nn.Linear(num_cells*hidden_size,vector_size)
        self.fc2 = nn.Linear(vector_size,num_classes)
        self.prelu = nn.PReLU()
        
    def forward(self, encoder_hidden):
        
        # when using a linear transformation the first axis should be batch 
        encoding_batch = encoder_hidden.view(-1,num_cells*hidden_size)
        vector_rep = self.prelu(self.fc1(encoding_batch))
        output = self.fc2(vector_rep)
        
        return output, vector_rep

In [9]:
def train(input_batches, input_lengths, targets, encoder, decoder, 
          encoder_optimizer, decoder_optimizer, criterion):
    
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word

    # Run words through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches)
    outputs, vector_rep = decoder(encoder_hidden)
    # Loss calculation and backpropagation
    loss = criterion(outputs,targets)
    
    #print(loss)
    
    loss.backward()

    # Update parameters with optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0] #, ec, dc

In [10]:
num_layers_encoder = 3
bidirectional = True
num_chars = len(char2idx)
hidden_size = 512
batch_size = 2
dropout = 0.0
vector_size = 256

encoder = EncoderRNN( num_chars, hidden_size, num_layers = num_layers_encoder, 
                     bidirectional = bidirectional, dropout = dropout)
if use_cuda:
    encoder = encoder.cuda()

input_batch, input_lengths, targets = random_batch(batch_size)

encoder_outputs, encoder_hidden = encoder(input_batch)

num_cells, batch_size, hidden_size = encoder_hidden.shape
print(num_cells, batch_size, hidden_size)

decoder = DecoderRNN(encoder_hidden,len(class2index),vector_size)
if use_cuda:
    decoder = decoder.cuda()

name = "_emo2_encode3L512_affine_256_bi_cpu"

encoder.load_state_dict(torch.load("modelstate/encoder"+name+".pth"))
decoder.load_state_dict(torch.load("modelstate/decoder"+name+".pth"))

6 2 512


In [154]:
# Begin!

# Keep track of time elapsed and running averages
start = time.time()
plot_losses = []
print_loss_total = 0 # Reset every print_every

# configure training # good params for the dummy set are bs = 4, 80 epochs, printevery 4, gamma = 0.9, lr = .001
batch_size = 4
n_epochs = 400
epoch = 0
print_every = 2 #10
gamma = .99
learning_rate = 0.001
vettedlist = [8,1,12,14,15,21,22,34,36,37,38,45,46,49,56,65,70,71,73,75]
# Initialize optimizers and criterion

encoder_optimizer = optim.Adam(encoder.parameters(),lr=learning_rate)

decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

escheduler = optim.lr_scheduler.StepLR(encoder_optimizer, step_size=print_every, gamma=gamma) 

dscheduler = optim.lr_scheduler.StepLR(decoder_optimizer, step_size=print_every, gamma=gamma) 

criterion = nn.CrossEntropyLoss()

while epoch < n_epochs:
    
    epoch += 1
    
    # Get training data for this cycle
    input_batches, input_lengths, targets = random_batch(batch_size)

    # Run the train function
    loss = train(input_batches, input_lengths, targets, encoder, decoder,
                 encoder_optimizer, decoder_optimizer, criterion)
    
    # Keep track of loss
    print_loss_total += loss 

    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), 
                                               epoch, epoch / n_epochs * 100, print_loss_avg)
        print(print_summary)
        input_var, labels = validationBatch(validation_pairs,vettedlist)
        val_acc = validationAccuracy(input_var, labels)
        print("val acc: ",val_acc)
        
    
    escheduler.step()
    dscheduler.step()
    
# 1/8 = 0.125, -ln(1/8) = 2.08
name = "_emo3_encode3L512_affine_256_bi_"

torch.save(encoder.state_dict(), "modelstate/encoder"+name+"cpu.pth")
torch.save(decoder.state_dict(), "modelstate/decoder"+name+"cpu.pth")


4m 2s (- 804m 16s) (2 0%) 2.4334
val acc:  0.0
7m 35s (- 751m 2s) (4 1%) 1.6498
val acc:  0.05
12m 21s (- 811m 51s) (6 1%) 2.3029
val acc:  0.05
15m 25s (- 755m 27s) (8 2%) 1.6697
val acc:  0.05
96m 1s (- 3744m 41s) (10 2%) 1.6317
val acc:  0.1
100m 16s (- 3242m 3s) (12 3%) 2.9125
val acc:  0.15
103m 33s (- 2855m 7s) (14 3%) 0.9803
val acc:  0.2


KeyboardInterrupt: 

In [12]:
name = "_emo3_encode3L512_affine_256_bi_"

torch.save(encoder.state_dict(), "modelstate/encoder"+name+"cpu.pth")
torch.save(decoder.state_dict(), "modelstate/decoder"+name+"cpu.pth")



In [21]:
def string2prediction(string):
    input_var = indexesFromSentence(string)
    input_var = Variable(torch.LongTensor(input_var)).unsqueeze(1) #.transpose(0, 1)#.unsqueeze(0) 
    if use_cuda:
        input_var = input_var.cuda()
    #print(input_var)
    encoder_outputs, encoder_hidden = encoder(input_var)
    #print(encoder_hidden.shape)
    outputs, vector_rep = decoder(encoder_hidden)
    return index2class[np.argmax(outputs.data.cpu().numpy())]

In [22]:
string = "*Phone Rings*\n\nMom: Get That. \n\nMe: Hello?\n\nSister: Hey, what's going on?\n\nMe: Oh, just fartin' around."
string2prediction(string)

'conversation'

In [20]:
215/604 

0.35596026490066224

In [17]:
count_dict = {}
for pair in validation_pairs:
    if pair[1] in count_dict:
        count_dict[pair[1]] += 1
    else:
        count_dict[pair[1]] = 0

In [18]:
count_dict # majority classifier = 215/604 = 0.35596

{'addiction': 36,
 'anxiety': 60,
 'autism': 13,
 'bipolar': 56,
 'conversation': 215,
 'depression': 201,
 'happy': 3,
 'schizophrenia': 12}

In [152]:
def validationBatch(validation_pairs, vettedlist):
    
    input_seqs = []
    targets = []
    #for pair in validation_pairs:
    for i in vettedlist:
        #input_seqs.append(indexesFromSentence(pair[0]))
        input_seqs.append(indexesFromSentence(validation_pairs[i][0]))
        targets.append(validation_pairs[i][1])
        
    # For input and target sequences, get array of lengths and pad with 0s to max length
    input_lengths = [len(s) for s in input_seqs]
    input_padded = [pad_seq(s, max(input_lengths)) for s in input_seqs]

    # Turn padded arrays into (batch_size x max_len) tensors, transpose into (max_len x batch_size)
    input_var = Variable(torch.LongTensor(input_padded)).transpose(0, 1)

    if use_cuda:
        input_var = input_var.cuda()
        
    labels = []
    for target in targets:
        labels.append(class2index[target])
        
    return input_var, labels

def validationAccuracy(input_var, labels):
        
    encoder_outputs, encoder_hidden = encoder(input_var)
    outputs, vector_rep = decoder(encoder_hidden)
    predictions = np.argmax(outputs.data.cpu().numpy(),axis=1)
    correct = np.array(labels) == predictions 
    correct =  correct + 0
    return np.sum(correct)/np.size(correct)

In [153]:
vettedlist = [8,1,12,14,15,21,22,34,36,37,38,45,46,49,56,65,70,71,73,75]
input_var, labels = validationBatch(validation_pairs,vettedlist)
val_acc = validationAccuracy(input_var, labels)
val_acc

0.0

0.05